# Flight Search Chatbot

Original Article: Build a Flight search chatbot from scratch using RASA Part 1 and 2 - Ashutosh Krishna, Medium 

Link: https://medium.com/datadriveninvestor/build-a-flight-search-chatbot-from-scratch-using-rasa-part-1-47370cf1e53b)

Source: https://github.com/ashukrishna100/MMTchatbot/tree/master/MMT/MMT

**Objectives** To build a chatbot capable of fetching latest info about the flights between 2 cities. It uses MakeMyTrip.com's website.

<img src="images/mmt.jpg">

## Rasa Stack

- Lets you focus on improving the “Chatbot” part of your project 
- Default set up of Rasa works really well right out of the box for intent extraction and dialogue management
- LOCAL models, no API calls for intent extraction and dialogue management, except if your business logic needs external calls.

## Architecture of chatbot

<img src="images/domain.png">


- As soon as Rasa receives a message from the end user, it tries to predict or extract the “intent” and “entities” present in the message. This part is handled by Rasa NLU
- Once the user’s intent is identified, the Rasa Stack performs an action called action_match_news to get the updates from the latest IPL match
- Rasa then tries to predict what it should do next. This decision is taken considering multiple factors and is handled by Rasa Core
- In this example, Rasa is showing the result of the most recent match to the user. It has also predicted the next action that our model should take – to check with the user whether the chatbot was able to solve his/her query

## Installations
Official Documentation: https://rasa.com/docs/core/installation/
* Python
* Rasa Starter Pack
* Spacy Language Model


### Python
* Install Anaconda 4.2.0 for Python 3.5 or Ananconda 5.2.0 for Python 3.6 
* Windows Build tools: Make sure the Microsoft $VC++$ Compiler Visual Studio 2015 is installed, so python can compile any dependencies or https://visualstudio.microsoft.com/visual-cpp-build-tools/ Download the installer and select VC++ Build tools in the list.

(My Note: on main page of Anaconda is https://www.anaconda.com/distribution/ you see Python 3.7, which is problematic, gives HTTP error while creating env. I had to delete 3.7 and get 3.6. Btw, this site does not mention which Anaconda installer has which Python version. WHY? thats the most crucial info. Anyway, I have given them here.)

### Python Environment
* By installing conda, you get base or the root environment, which is the default.
* Practical tip: DO NOT install any packages in the root. ALWAYS create and env and install inside the new env.
* Env is needed esepcially for fragile packages like Python (its treated as a package) and rasa.
* So, **conda create -n rasa python=3.6** 

(My Note: Env management is again a sour point. It creates complete copy (deeeep) of python 3.6 and all other packages inside "envs" folder. Goes to 1.5 GB!! Can someone optimize it?)


### MMT (Make My Trip) bot code (for reference)

* Clone repo from  https://github.com/ashukrishna100/MMTchatbot/tree/master/MMT/MMT 

* Instalation requirements.txt file can be take from https://github.com/mohdsanadzakirizvi/iplbot.git

* Additionally you will need to install Beautiful Soup

<!---
* Let’s take a look at the folder structure and the files that were created 
<img src="images/nlu.png">
* Install git, can git clone above package.
-->

* **WE WILL BE CODING IT FROM SCRATCH**
* Run following commands:

* **mkdir code; cd code;mkdir data**
* **activate rasa**
* **pip install -r iplbot_requirements.txt**
* **pip install bs4**

### Spacy Language Models
* Open cmd in Administrator mode and activate the env
* Download medium english by **python -m spacy download en_core_web_md** 
* Link with **python -m spacy link en_core_web_md en --force**


(My Note: First command will show that linking is done, but it may not have happened actually. So, do next step anyway.)

### Others
* Install nb_conda_kernels to let Jupyter Notebook see the new env
* We are not using API to access Make My Trip.com's data, but just the search results, like an usual user. Then Beautiful Soup library is used to parse the html results page

<!---
* Need Rasa Core SDK for custom actions **pip install rasa_core_sdk** and run it as **python -m rasa_core_sdk.endpoint --actions actions** where actions.py has all the custom actions
* Install pygraphviz by **conda install -c alubbock pygraphviz**
-->

### Importing Libraries

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

import rasa_nlu
import rasa_core
import spacy

## Rasa NLU: Extracting User Intent from a Message

The first thing we want to do is figure out the intent of the user. What does he or she want to accomplish? Let’s utilize Rasa and build an NLU model to identify user intent and its related entities.

### Preparing the NLU Training Data

Training data for extracting the user intent.
As you can see, the format of training data for ‘intent’ is quite simple in Rasa. You just have to:

- Start the line with “## intent:intent_name”
- Supply all the examples in the following lines

In [2]:
nlu_md = """
## intent:flight
-hey! I want latest flight schedule
-can you provide me flight schedule
-please provide latest flight info
-is it possible to get latest flight information
-flight schedule
-flight information
-flight info


## intent:inform
-[DEL](location)
-[BOM](location)
-[CCU](location)
-[BLR](location)
-[HYD](location)
-[MAA](location)
-[PNQ](location)
-[IXC](location)
-[09-01-2019](date)
- I want for [09-02-2019](date)
- i want to travel on [21-04-2019](date)


## intent:affirmation
-ok
-yea
-yes
-definitely
-yah

## intent:deny
-no
-not yet
-never
-not now
"""

%store nlu_md > data/nlu.md

Writing 'nlu_md' (str) to file 'data/nlu.md'.


You can include as many examples as you want for each intent. In fact, make sure to include slangs and short forms that you use while texting. The idea is to make the chatbot understand the way we type text. Feel free to refer to the complete version where I have given plenty of examples for each intent type.

### Defining the NLU Model Configuration

This file lets us create a text processing pipeline in Rasa. Luckily for us, Rasa comes with two default settings based on the amount of training data we have:
- “spacy_sklearn” pipeline if you have less than 1000 training examples
- “tensorflow_embedding” if you have a large amount of data

In [3]:
config = """
language: "en"

pipeline:
- name : "nlp_spacy"
- name: "tokenizer_whitespace"
- name: "intent_entity_featurizer_regex"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
  intent_tokenization_flag: true
  intent_split_symbol: "+" 
""" 

%store config > config/config.yml

Writing 'config' (str) to file 'config/config.yml'.


### Training the NLU Classifier Model

On command line you can run following command:

**python -m rasa_nlu.train -c config/config.yml --data data/nlu.md -o models --fixed_model_name nlu --project current --verbose**

Or programmatically you can write code

In [4]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("data/nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config/config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="chatter")

INFO:rasa_nlu.training_data.loading:Training data format of data/nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 27 (4 distinct intents)
	- Found intents: 'inform', 'deny', 'affirmation', 'flight'
	- entity examples: 11 (2 distinct entities)
	- found entities: 'location', 'date'

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:rasa_nlu.model:Starting to train component nlp_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component tokenizer_whitespace
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_entity_featurizer_regex
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_crf
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_syn

### Evaluating the NLU model on a random text (first way)

Let’s test how good our model is performing by giving it a sample text that it hasn’t been trained on for extracting intent. 

In [5]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("please provide latest flight info"))

{
  "intent": {
    "name": "flight",
    "confidence": 0.9705340266227722
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "flight",
      "confidence": 0.9705340266227722
    },
    {
      "name": "affirmation",
      "confidence": 0.0
    },
    {
      "name": "deny",
      "confidence": 0.0
    },
    {
      "name": "inform",
      "confidence": 0.0
    }
  ],
  "text": "please provide latest flight info"
}


Not only does our NLU model perform well on intent extraction, but it also ranks the other intents based on their confidence scores. This is a nifty little feature that can be really useful when the classifier is confused between multiple intents.

### Evaluating the NLU model on a random text (2nd way)
Let’s test how good our model is performing by giving it a sample text that it hasn’t been trained on for extracting intent. You can open an iPython/Python shell and follow the following steps:

In [6]:
from rasa_nlu.model import Interpreter
nlu_model = Interpreter.load('./models/nlu/default/chatter')
nlu_model.parse('please provide latest flight info')

INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:tensorflow:Restoring parameters from ./models/nlu/default/chatter\intent_classifier_tensorflow_embedding.ckpt


{'intent': {'name': 'flight', 'confidence': 0.9705340266227722},
 'entities': [],
 'intent_ranking': [{'name': 'flight', 'confidence': 0.9705340266227722},
  {'name': 'affirmation', 'confidence': 0.0},
  {'name': 'deny', 'confidence': 0.0},
  {'name': 'inform', 'confidence': 0.0}],
 'text': 'please provide latest flight info'}

### Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [7]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("data/nlu.md", model_directory)

INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:tensorflow:Restoring parameters from C:\YogeshKulkarni\Training\TeachingDataScience\Jupyter\bookingbot\./models/nlu\default\chatter\intent_classifier_tensorflow_embedding.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of data/nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 27 (4 distinct intents)
	- Found intents: 'inform', 'deny', 'affirmation', 'flight'
	- entity examples: 11 (2 distinct entities)
	- found entities: 'location', 'date'

INFO:rasa_nlu.evaluate:Intent evaluation results:
INFO:rasa_nlu.evaluate:Intent Evaluation: Only considering those 27 examples that have a defined intent out of 27 examples
INFO:rasa_nlu.evaluate:F1-Score:  1.0
INFO:rasa_nlu.evaluate:Precision: 1.0
INFO:rasa_nlu.evaluate:Accuracy:  1.0
INFO:rasa_nlu.evaluate:Classification report: 
              precision    recall  f1-score   support

 affirmation     

{'intent_evaluation': {'predictions': [{'text': 'hey! I want latest flight schedule',
    'intent': 'flight',
    'predicted': 'flight',
    'confidence': 0.9746279716491699},
   {'text': 'can you provide me flight schedule',
    'intent': 'flight',
    'predicted': 'flight',
    'confidence': 0.9729326963424683},
   {'text': 'please provide latest flight info',
    'intent': 'flight',
    'predicted': 'flight',
    'confidence': 0.9705340266227722},
   {'text': 'is it possible to get latest flight information',
    'intent': 'flight',
    'predicted': 'flight',
    'confidence': 0.9685079455375671},
   {'text': 'flight schedule',
    'intent': 'flight',
    'predicted': 'flight',
    'confidence': 0.9815781116485596},
   {'text': 'flight information',
    'intent': 'flight',
    'predicted': 'flight',
    'confidence': 0.9756697416305542},
   {'text': 'flight info',
    'intent': 'flight',
    'predicted': 'flight',
    'confidence': 0.9651832580566406},
   {'text': 'DEL',
    'intent

## Rasa Core: Making Interactive Conversations

One of the most important aspects of a chatbot application is its ability to be interactive. 

### Designing the conversational flow

Think of the simplest conversation our chatbot can have with a user. What would be the flow of such a conversation?

---
Me: Hi

Iplbot: Hey! How may I help you?

Me: Please provide me latest flight info

bot: Please provide your origin airport code?

Me: PNQ

bot: Please provide your destination airport code?

Me: BLR

bot: What is the date of your travel (in dd-mm-yyyy)?

Me: 27-05-2019

bot: Here is the list of carriers with their fare

Jet Airways : Rs. 8,220
Vistara : Rs. 14,570
IndiGo: Rs. 5,932

bot: Do you want to make another inquiry?

Me: no

bot: Thanks for contacting us. Have a good day!

---
Let’s see how we can teach a simple conversation like that to Rasa:


The general format is:

This is called a user story path. I have provided a few stories in the data/stories.md file for your reference. This is the training data for Rasa Core.

### Writing  Stories

The way it works is:

- Give some examples of sample story paths that the user is expected to follow
- Rasa Core combines them randomly to create more complex user paths
- It then builds a probabilistic model out of that. This model is used to predict the next action Rasa should take

In [8]:
stories_md = """
## fallback
- utter_unclear


## Story 1
* flight
    - utter_boarding
* inform{"location": "BOM"}
    - action_save_origin
    - slot{"from": "BOM"}
    - utter_destination
* inform{"location": "DEL"}
    - action_save_destination
    - slot{"to": "DEL"}
    - utter_date
* inform{"date": "20-01-2019"}
    - action_save_date
    - slot{"date": "20-01-2019"}
    - utter_confirm
* affirmation
    - action_get_flight
	- utter_check_another_one
* deny
	- utter_thanks
	- action_restart

## Stry 2-multiple steps
* flight
    - utter_boarding
* inform{"location": "PNQ"}
    - action_save_origin
    - slot{"from": "PNQ"}
    - utter_destination
* inform{"location": "BLR"}
    - action_save_destination
    - slot{"to": "BLR"}
    - utter_date
* inform{"date": "03-02-2019"}
    - slot{"date": "03-02-2019"}
    - action_save_date
    - slot{"date": "03-02-2019"}
    - utter_confirm
* affirmation
    - action_get_flight
    - utter_check_another_one
* affirmation
	- action_slot_reset
	- reset_slots
    - utter_boarding
* inform{"location": "BOM"}
    - action_save_origin
    - slot{"from": "BOM"}
    - utter_destination
* inform{"location": "DEL"}
    - action_save_destination
    - slot{"to": "DEL"}
    - utter_date
* inform{"date": "10-02-2019"}
    - slot{"date": "10-02-2019"}
    - action_save_date
    - slot{"date": "10-02-2019"}
    - utter_confirm
* affirmation
    - action_get_flight
    - utter_check_another_one
* deny
    - utter_thanks
    - action_restart
"""

%store stories_md > data/stories.md

Writing 'stories_md' (str) to file 'data/stories.md'.


Now, generate a similar graph for your stories using the following command:

**python -m rasa_core.visualize -d domain.yml -s data/stories.md -o graph.html**

This is very helpful when debugging the conversational flow of the chatbot.

### Defining the Domain

The domain is the world of your chatbot. It contains everything the chatbot should know, including:

- All the actions it is capable of doing
- The intents it should understand
- The template of all the utterances it should tell the user, and much more

In [9]:
domain_yml = """
intents:
- ticket
- inform
- affirmation
- deny

entities:
- location 
- date

slots:
   from:
            type: text
   to:
            type: text
   date:
            type: text

actions:
- utter_boarding
- utter_destination
- utter_date
- utter_confirm
- utter_check_another_one
- utter_thanks
- utter_unclear
- action_save_origin
- action_save_destination
- action_save_date
- action_get_flight
- action_slot_reset
templates:
  
  utter_boarding:
  - text: "We'll help you find the latest flight schedule. First, please provide your origin airport code?"
  
  utter_destination:
  - text: "And the destination airport code?"
  
  utter_date:
  - text: "What is the date for your travel(in dd-mm-yyyy)?"
  
  utter_confirm:
  - text: "I will be making inquiry for flight from {from} to {to} on {date}. Is that correct?"
  
  utter_check_another_one:
  - text: "Do you want to make another inquiry?"
  
  utter_thanks:
  - text: "Thanks for contacting us. Have a good day!"
  
  utter_unclear:
  - text: "I am not sure what you are aiming for.Kindly try it again"
"""

%store domain_yml > config/domain.yml

Writing 'domain_yml' (str) to file 'config/domain.yml'.


### Setting Policies

Rasa Core generates the training data for the conversational part using the stories we provide. It also lets you define a set of policies to use when deciding the next action of the chatbot. These policies are defined in the policies.yml file.

In [10]:
policies_yml = """
policies:
  - name: KerasPolicy
    epochs: 100
    max_history: 5
  - name: FallbackPolicy
    fallback_action_name: 'action_default_fallback'
  - name: MemoizationPolicy
    max_history: 5
"""

%store policies_yml > config/policies.yml

Writing 'policies_yml' (str) to file 'config/policies.yml'.


Notes:
- KerasPolicy uses a neural network implemented in Keras to select the next action. The default architecture is based on an LSTM (Long Short Term Memory) model
- MemoizationPolicy memorizes the conversations in your training data. It predicts the next action with confidence 1.0 if this exact conversation exists in the training data, otherwise, it predicts ‘None’ with confidence 0.0
- FallbackPolicy invokes a fallback action if the intent recognition has confidence below nlu_threshold or if none of the dialogue policies predict action with confidence higher than core_threshold
- One important hyperparameter for Rasa Core policies is the max_history. This controls how much dialogue history the model looks at to decide which action to take next

###  Custom Actions

Using Make My Trip search results for fetching flight related info. 

In [11]:
actions_py="""
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_core.domain import Domain
from rasa_core.trackers import EventVerbosity

import logging
logger = logging.getLogger(__name__)

import requests
import json

from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet
from rasa_core_sdk.events import UserUtteranceReverted
from rasa_core_sdk.events import AllSlotsReset
from rasa_core_sdk.events import Restarted


		 
class SaveOrigin(Action):
	def name(self):
		return 'action_save_origin'
		
	def run(self, dispatcher, tracker, domain):
		orig = next(tracker.get_latest_entity_values("location"), None)
		if not orig:
			dispatcher.utter_message("Please enter a valid airport code")
			return [UserUtteranceReverted()]
		return [SlotSet('from',orig)]
	


class SaveDestination(Action):
	def name(self):
		return 'action_save_destination'
		
	def run(self, dispatcher, tracker, domain):
		dest = next(tracker.get_latest_entity_values("location"), None)
		if not dest:
			dispatcher.utter_message("Please enter a valid airport code")
			return [UserUtteranceReverted()]
		return [SlotSet('to',dest)]
		
		
class SaveDate(Action):
	def name(self):
		return 'action_save_date'
		
	def run(self, dispatcher, tracker, domain):
		inp = next(tracker.get_latest_entity_values("date"), None)
		if not inp:
			dispatcher.utter_message("Please enter a valid date")
			return [UserUtteranceReverted()]
		return [SlotSet('date',inp)]
		
class ActionSlotReset(Action): 	
    def name(self): 		
        return 'action_slot_reset' 	
    def run(self, dispatcher, tracker, domain): 		
        return[AllSlotsReset()]
		

		
from bs4 import BeautifulSoup
import urllib.request
import re


class getFlightStatus(Action):
	def name(self):
		return 'action_get_flight'
	def run(self, dispatcher, tracker, domain):
		orig=tracker.get_slot('from')
		dest=tracker.get_slot('to')
		dat=tracker.get_slot('date')
		quote_page = "https://flights.makemytrip.com/makemytrip/search/O/O/E/1/0/0/S/V0/{}_{}_{}?contains=false&remove="
		page=urllib.request.urlopen(quote_page.format(orig,dest,dat))
		soup = BeautifulSoup(page, 'html.parser')
		list1=[]
		message=soup.find_all('label',attrs={'class':'flL mtop5 mleft3 vallabel'})
		dispatcher.utter_message("Here is the list of carriers with their fare")
		for a in message:
			list1.append(a.text)	
		message1=soup.find_all('span',attrs={'class':'flR'})
		list2=[]
		for b in message1:
			if "Rs." in b.text:
				list2.append(re.sub('\s+', '', b.text))
		for i in range(len(list1)):
			dispatcher.utter_message(list1[i]+" : "+list2[i])
		return []
"""
%store actions_py > actions.py

Writing 'actions_py' (str) to file 'actions.py'.


Need endpoints yml to execute the actions server.

Note: If you have external API call, like REST, need to have "webhook" word at the end, else nothing.

My own query on this topic: https://forum.rasa.com/t/rasa-core-sdk-not-working/9228

In [12]:
endpoints_yml = """
#action_endpoint:
#  url: "http://localhost:5055/webhook"
  
action_endpoint:
  url: http://localhost:5055/webhook

#nlg:
#  url: http://localhost:5056/nlg

core_endpoint:
  url: http://localhost:5005
"""
%store endpoints_yml > endpoints.yml

Writing 'endpoints_yml' (str) to file 'endpoints.yml'.


In a separate shell (cmd for Windows):
- **activate rasa**
- come to directory where actions.py is and then run
- **python -m rasa_core_sdk.endpoint --actions actions**

This way, custom action server starts ...

In [19]:
# import sys
# python = sys.executable
# !{python} -m rasa_core_sdk.endpoint --actions actions

###  Visualising the Training Data

In [21]:

from IPython.display import Image, display
from rasa_core.agent import Agent
%matplotlib inline

agent = Agent('config/domain.yml')
agent.visualize("data/stories.md", "images/story_graph.png", max_history=2)
i = Image(filename="images/story_graph.png")
display(i)

Processed Story Blocks: 100%|█████| 3/3 [00:00<00:00, 374.98it/s, # trackers=1]


See image as

<img src="images/story_graph.png" />

### Training a Dialogue Model

You can train the model using the following command:

**python -m rasa_core.train -d config/domain.yml -s data/stories.md -o models/current/dialogue -c config/policies.yml**

Or do it programmatically as:

In [24]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

# agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

agent = Agent('config/domain.yml', policies = [MemoizationPolicy(max_history=2), KerasPolicy(validation_split=0.2,epochs=200)])
# loading our neatly defined training dialogues
training_data = agent.load_data('data/stories.md')

agent.train(training_data)

# FOLLOWING WAY HAS BEEN DEPRECATED, pass these parameters to KerasPolicy above
# agent.train(
#     training_data,
#     validation_split=0.0,
#     epochs=200
# )

agent.persist('models/current/dialogue')

Processed Story Blocks: 100%|█████| 3/3 [00:00<00:00, 199.99it/s, # trackers=6]
Processed actions: 37it [00:00, 739.96it/s, # examples=37]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 28)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7808      
_________________________________________________________________
dense (Dense)                (None, 19)                627       
_________________________________________________________________
activation (Activation)      (None, 19)                0         
Total params: 8,435
Trainable params: 8,435
Non-trainable params: 0
_________________________________________________________________


INFO:rasa_core.policies.keras_policy:Fitting model with 60 total samples and a validation split of 0.2


Epoch 1/200
60/60 [==============================] - ETA: 0s - loss: 2.9537 - acc: 0.125 - 1s 17ms/step - loss: 2.9407 - acc: 0.1333
Epoch 2/200
60/60 [==============================] - ETA: 0s - loss: 2.9007 - acc: 0.156 - 0s 327us/step - loss: 2.8897 - acc: 0.1833
Epoch 3/200
60/60 [==============================] - ETA: 0s - loss: 2.8392 - acc: 0.250 - 0s 883us/step - loss: 2.8372 - acc: 0.2333
Epoch 4/200
60/60 [==============================] - ETA: 0s - loss: 2.8130 - acc: 0.406 - 0s 333us/step - loss: 2.8147 - acc: 0.4167
Epoch 5/200
60/60 [==============================] - ETA: 0s - loss: 2.7859 - acc: 0.406 - 0s 267us/step - loss: 2.7831 - acc: 0.4167
Epoch 6/200
60/60 [==============================] - ETA: 0s - loss: 2.7482 - acc: 0.343 - 0s 250us/step - loss: 2.7495 - acc: 0.3667
Epoch 7/200
60/60 [==============================] - ETA: 0s - loss: 2.6937 - acc: 0.343 - 0s 233us/step - loss: 2.7133 - acc: 0.3833
Epoch 8/200
60/60 [==============================] - ETA: 0s - 

Epoch 62/200
60/60 [==============================] - ETA: 0s - loss: 1.5270 - acc: 0.437 - 0s 267us/step - loss: 1.6603 - acc: 0.4167
Epoch 63/200
60/60 [==============================] - ETA: 0s - loss: 1.5551 - acc: 0.437 - 0s 267us/step - loss: 1.6704 - acc: 0.4167
Epoch 64/200
60/60 [==============================] - ETA: 0s - loss: 1.4941 - acc: 0.437 - 0s 317us/step - loss: 1.5779 - acc: 0.4167
Epoch 65/200
60/60 [==============================] - ETA: 0s - loss: 1.4701 - acc: 0.437 - 0s 283us/step - loss: 1.5997 - acc: 0.4167
Epoch 66/200
60/60 [==============================] - ETA: 0s - loss: 1.4519 - acc: 0.437 - 0s 300us/step - loss: 1.5935 - acc: 0.4167
Epoch 67/200
60/60 [==============================] - ETA: 0s - loss: 1.4190 - acc: 0.437 - 0s 333us/step - loss: 1.5671 - acc: 0.4167
Epoch 68/200
60/60 [==============================] - ETA: 0s - loss: 1.4585 - acc: 0.437 - 0s 317us/step - loss: 1.5836 - acc: 0.4167
Epoch 69/200
60/60 [==============================] - E

60/60 [==============================] - ETA: 0s - loss: 0.5663 - acc: 0.906 - 0s 300us/step - loss: 0.6300 - acc: 0.9000
Epoch 183/200
60/60 [==============================] - ETA: 0s - loss: 0.5101 - acc: 0.968 - 0s 333us/step - loss: 0.6417 - acc: 0.8833
Epoch 184/200
60/60 [==============================] - ETA: 0s - loss: 0.5282 - acc: 0.968 - 0s 283us/step - loss: 0.5730 - acc: 0.9500
Epoch 185/200
60/60 [==============================] - ETA: 0s - loss: 0.5773 - acc: 0.843 - 0s 367us/step - loss: 0.5786 - acc: 0.8667
Epoch 186/200
60/60 [==============================] - ETA: 0s - loss: 0.5958 - acc: 0.906 - 0s 350us/step - loss: 0.6494 - acc: 0.8833
Epoch 187/200
60/60 [==============================] - ETA: 0s - loss: 0.5778 - acc: 0.812 - 0s 367us/step - loss: 0.6066 - acc: 0.8500
Epoch 188/200
60/60 [==============================] - ETA: 0s - loss: 0.5452 - acc: 0.906 - 0s 333us/step - loss: 0.6144 - acc: 0.9000
Epoch 189/200
60/60 [==============================] - ETA: 0s

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/current/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to 'D:\Yogesh\ToDos\Projects\Teaching\Teaching_DataScience\Jupyter\bookingbot\models\current\dialogue'


## Running: Talk to your Bot

So we have the chatbot ready. It’s time to chat

One way is to run following command in shell (windows cmd)
- activate the environment,
- come to directory where actions.py is and then run
- **python -m rasa_core.run -d models/current/dialogue -u models/current/nlu --endpoints endpoints.yml**

Or else do programmatically like below

Both approaches expect rasa core sdk server running in a separate window, else **python -m rasa_core_sdk.endpoint --actions actions**

In [25]:
# #Starting the Bot

# from rasa_core.agent import Agent
# agent = Agent.load('models/current/dialogue', interpreter=model_directory)

import IPython
from IPython.display import clear_output
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
from rasa_core.utils import EndpointConfig
import time

def load_assistant():
    messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
    interpreter = NaturalLanguageInterpreter.create(model_directory)
    endpoint = EndpointConfig('http://localhost:5055/webhook')
    agent = Agent.load('models/current/dialogue', interpreter=interpreter, action_endpoint = endpoint)

    print("start the conversation here or send 'stop'")
    print()
    print("Hi! How Can I help you today?")
    while True:
        a = input()
        if a == 'stop':
            break
        responses = agent.handle_text(a)
        for response in responses:
            print(response["text"])

In [ ]:
load_assistant()        

INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.


INFO:tensorflow:Restoring parameters from D:\Yogesh\ToDos\Projects\Teaching\Teaching_DataScience\Jupyter\bookingbot\./models/nlu\default\chatter\intent_classifier_tensorflow_embedding.ckpt


INFO:tensorflow:Restoring parameters from D:\Yogesh\ToDos\Projects\Teaching\Teaching_DataScience\Jupyter\bookingbot\./models/nlu\default\chatter\intent_classifier_tensorflow_embedding.ckpt


Your bot is ready to talk! Type your messages here or send 'stop'
Hi
flight info
We'll help you find the latest flight schedule. First, please provide your origin airport code?
PNQ
And the destination airport code?
BLR
What is the date for your travel(in dd-mm-yyyy)?
29-05-2019
I will be making inquiry for flight from PNQ to BLR on 29-05-2019. Is that correct?
Yes
Here is the list of carriers with their fare
Spicejet : Rs.6,218
IndiGo : Rs.3,876
Do you want to make another inquiry?
No
Thanks for contacting us. Have a good day!


## What Next?

- Try to use different pipelines in Rasa Core, explore more Policies, fine-tune those models, 
- Check out what other features Make My Trip or any other similar site provides, etc.
- Other APIs
- Different languages (Hindi bot?)

